In [4]:
import os
import rasterio
import numpy as np
from scipy.ndimage import distance_transform_edt

# Paths to water probability rasters (skip 2024 since not available)
water_rasters = {
    1990: r"H:\Major_Project\Work\lpicodes\Brahmaputra_Input\BRAHMAPUTRA_1990.tif",
    2000: r"H:\Major_Project\Work\lpicodes\Brahmaputra_Input\BRAHMAPUTRA_2000.tif",
    2010: r"H:\Major_Project\Work\lpicodes\Brahmaputra_Input\BRAHMAPUTRA_2010.tif",
    2020: r"H:\Major_Project\Work\lpicodes\Brahmaputra_Input\BRAHMAPUTRA_2020.tif"
    # 2024 intentionally excluded
}

# Output folder
output_dir = r"H:\Major_Project\Work\Recovery_Index_Training\Output_1"

# Threshold to define water (adjust if needed)
threshold = 0.5  # If your data is 0–100, set this to 50

# Pixel size (in meters)
pixel_size = 30  # meters

def compute_distance_to_water(water_path, output_path, threshold=0.5, pixel_size=30):
    with rasterio.open(water_path) as src:
        water_prob = src.read(1).astype('float32')
        profile = src.profile

    # Normalize if in 0–100 scale
    if water_prob.max() > 1:
        water_prob /= 100.0

    # Create binary water mask: 1 = water, 0 = land
    binary_water = (water_prob > threshold).astype(np.uint8)

    # Compute Euclidean distance to water (in pixels)
    distance_pixels = distance_transform_edt(binary_water == 0)

    # Convert distance from pixels to meters
    distance_meters = distance_pixels * pixel_size

    # Normalize distance to the 0-1 range
    min_distance = distance_meters.min()
    max_distance = distance_meters.max()
    
    # If there's variation in the distance raster (min != max), normalize
    if max_distance > min_distance:
        normalized_distance = (distance_meters - min_distance) / (max_distance - min_distance)
    else:
        # If there's no variation, set the entire raster to 0 (since the distance is the same)
        normalized_distance = np.zeros_like(distance_meters)

    # Save normalized distance raster
    profile.update(dtype='float32', count=1, nodata=None)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(normalized_distance.astype('float32'), 1)

    print(f"✅ Saved: {output_path}")

# Run for each year where data is available
for year, input_path in water_rasters.items():
    output_path = os.path.join(output_dir, f"normalized_distance_to_water_{year}.tif")
    compute_distance_to_water(input_path, output_path, threshold, pixel_size)



✅ Saved: H:\Major_Project\Work\Recovery_Index_Training\Output_1\normalized_distance_to_water_1990.tif
✅ Saved: H:\Major_Project\Work\Recovery_Index_Training\Output_1\normalized_distance_to_water_2000.tif
✅ Saved: H:\Major_Project\Work\Recovery_Index_Training\Output_1\normalized_distance_to_water_2010.tif
✅ Saved: H:\Major_Project\Work\Recovery_Index_Training\Output_1\normalized_distance_to_water_2020.tif
